In [472]:
def infer_power(sample_sizes_to_eval, sub_iterations, meta_iterations,
                p_value_threshold, desired_power, data):
    import pandas as pd
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    import numpy as np
    from scipy.optimize import curve_fit
    
    meta_results        = []
    meta_supplement     = []
    sub_iterations_list = []
    
    df = pd.read_csv(data, index_col=False)
    df = df[['Customer_ID', 'Retailer_ID', 'Treated', 'Order_Amt']]
    print(str(len(df)) + ' observations read into Python.')
    
    master_counter = 0
    while master_counter < meta_iterations:
    
        master_results = []
        for k in sample_sizes_to_eval:
            counter = 0 
            pvalues = []
            while counter < sub_iterations:
                working_df = df.sample(k, replace=False)
                X = working_df.Treated
                X = sm.add_constant(X)
                Y = working_df.Order_Amt
                model = sm.OLS(Y, X).fit(method='pinv').get_robustcov_results(
                        'cluster', groups = working_df.Customer_ID, 
                        use_correction=True, df_correction=True)
                pvalues.append(model.pvalues[1])
                counter += 1
  
            results = []
            for i in pvalues:
                if i <= p_value_threshold:
                    results.append(1)    
                else:
                    results.append(0)  
            out = []
            out.append(k)
            out.append(p_value_threshold)
            out.append(sum(results)/len(results)) 
            master_results.append(out)

        def exp_func(x, a, b, c):
            return a * np.log(b * x) + c

        eta = []
        for i in master_results:
            eta.append(i[0])
        eta = np.asarray(eta)

        cdf = []
        for i in master_results:
            cdf.append(i[2])
        cdf = np.asarray(cdf)

        popt, pcov = curve_fit(exp_func, eta, cdf)

        recommended_n = int(np.exp((desired_power - popt[2])/popt[0])/popt[1])
        print("The recommended sample size for a statistical power level of " +
              "{} is {}.".format(desired_power, recommended_n))

        final_pvalues = []
        counter_2 = 0 
        while counter_2 < sub_iterations:
            working_df_2 = df.sample(recommended_n, replace=False)
            X = working_df_2.Treated
            X = sm.add_constant(X)
            Y = working_df_2.Order_Amt
            model_2 = sm.OLS(Y, X).fit(method='pinv').get_robustcov_results(
                      'cluster', groups = working_df_2.Customer_ID, 
                      use_correction=True, df_correction=True)
            final_pvalues.append(model_2.pvalues[1])
            counter_2 += 1

        final_results = []
        for i in final_pvalues:
            if i <= p_value_threshold:
                final_results.append(1)    
            else:
                final_results.append(0)  

        meta_results.append(sum(final_results)/len(final_results))
        meta_supplement.append(desired_power)
        sub_iterations_list.append(sub_iterations)
        master_counter += 1
        
    df_out = pd.DataFrame(list(zip(meta_supplement, meta_results))) 
    df_out.columns = ['Desired Power', 'Actual Power']
    df_out['Error'] = (df_out['Desired Power'] - df_out['Actual Power']).abs()
    df_out['Iterations'] = sub_iterations_list
    return df_out

In [473]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math

In [474]:
results_sd_3_n_500 = infer_power([50000, 70000, 90000], 500, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_3_n_500 complete.')
mu         = results_sd_3_n_500['Error'].mean()
sigma      = results_sd_3_n_500['Error'].std()
iterations = results_sd_3_n_500['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('50k_70k_90k_n500.png')
plt.clf()

139793 observations read into Python.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 8778.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 2912.


...

/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 9365.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 8578.
results_sd_3_n_500 complete.


<Figure size 432x288 with 0 Axes>

In [475]:
results_sd_3_n_1000 = infer_power([50000, 70000, 90000], 1000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_3_n_1000 complete.')
mu         = results_sd_3_n_1000['Error'].mean()
sigma      = results_sd_3_n_1000['Error'].std()
iterations = results_sd_3_n_1000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)

bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('50k_70k_90k_n1000.png')
plt.clf()

139793 observations read into Python.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 4175.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 5548.


...

/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 1886.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 2373.
results_sd_3_n_1000 complete.


<Figure size 432x288 with 0 Axes>

In [ ]:
results_sd_3_n_2000 = infer_power([50000, 70000, 90000], 2000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_3_n_2000 complete.')
mu         = results_sd_3_n_2000['Error'].mean()
sigma      = results_sd_3_n_2000['Error'].std()
iterations = results_sd_3_n_2000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('50k_70k_90k_n2000.png')
plt.clf()

139793 observations read into Python.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 6276.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 6852.


...

/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 4488.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The recommended sample size for a statistical power level of 0.8 is 4826.
results_sd_3_n_2000 complete.


<Figure size 432x288 with 0 Axes>

In [ ]:
results_sd_5_n_500 = infer_power([30000, 50000, 70000, 90000, 110000], 
                                  500, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_5_n_500 complete.')
mu         = results_sd_5_n_500['Error'].mean()
sigma      = results_sd_5_n_500['Error'].std()
iterations = results_sd_5_n_500['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('30k_50k_70k_90k_11k_n500.png')
plt.clf()

139793 observations read into Python.
The recommended sample size for a statistical power level of 0.8 is 17433.
The recommended sample size for a statistical power level of 0.8 is 16947.
The recommended sample size for a statistical power level of 0.8 is 21082.
The recommended sample size for a statistical power level of 0.8 is 12743.
The recommended sample size for a statistical power level of 0.8 is 13595.
The recommended sample size for a statistical power level of 0.8 is 18571.
The recommended sample size for a statistical power level of 0.8 is 13014.
The recommended sample size for a statistical power level of 0.8 is 14999.
The recommended sample size for a statistical power level of 0.8 is 13273.
The recommended sample size for a statistical power level of 0.8 is 18047.
The recommended sample size for a statistical power level of 0.8 is 19522.
The recommended sample size for a statistical power level of 0.8 is 18422.
The recommended sample size for a statistical power level of 0

<Figure size 432x288 with 0 Axes>

In [ ]:
results_sd_5_n_1000 = infer_power([30000, 50000, 70000, 90000, 110000], 
                                   1000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_5_n_1000 complete.')
mu         = results_sd_5_n_1000['Error'].mean()
sigma      = results_sd_5_n_1000['Error'].std()
iterations = results_sd_5_n_1000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('30k_50k_70k_90k_11k_n1000.png')
plt.clf()

139793 observations read into Python.
The recommended sample size for a statistical power level of 0.8 is 16944.
The recommended sample size for a statistical power level of 0.8 is 14962.
The recommended sample size for a statistical power level of 0.8 is 16098.
The recommended sample size for a statistical power level of 0.8 is 14973.
The recommended sample size for a statistical power level of 0.8 is 17538.
The recommended sample size for a statistical power level of 0.8 is 18635.
The recommended sample size for a statistical power level of 0.8 is 13259.
The recommended sample size for a statistical power level of 0.8 is 13831.
The recommended sample size for a statistical power level of 0.8 is 15968.
The recommended sample size for a statistical power level of 0.8 is 15447.
The recommended sample size for a statistical power level of 0.8 is 15571.
The recommended sample size for a statistical power level of 0.8 is 13477.
The recommended sample size for a statistical power level of 0

<Figure size 432x288 with 0 Axes>

In [ ]:
results_sd_5_n_2000 = infer_power([30000, 50000, 70000, 90000, 110000], 
                                   2000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_5_n_1000 complete.')
mu         = results_sd_5_n_2000['Error'].mean()
sigma      = results_sd_5_n_2000['Error'].std()
iterations = results_sd_5_n_2000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('30k_50k_70k_90k_11k_n2000.png')
plt.clf()

139793 observations read into Python.
The recommended sample size for a statistical power level of 0.8 is 14974.
The recommended sample size for a statistical power level of 0.8 is 15918.
The recommended sample size for a statistical power level of 0.8 is 16113.
The recommended sample size for a statistical power level of 0.8 is 15036.
The recommended sample size for a statistical power level of 0.8 is 17752.
The recommended sample size for a statistical power level of 0.8 is 15766.
The recommended sample size for a statistical power level of 0.8 is 13714.
The recommended sample size for a statistical power level of 0.8 is 17384.
The recommended sample size for a statistical power level of 0.8 is 16288.
The recommended sample size for a statistical power level of 0.8 is 16205.
The recommended sample size for a statistical power level of 0.8 is 16399.
The recommended sample size for a statistical power level of 0.8 is 17468.
The recommended sample size for a statistical power level of 0

In [ ]:
results_sd_7_n_500 = infer_power([10000, 30000, 50000, 70000, 90000, 110000, 130000], 
                                  500, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_7_n_500 complete.')
mu         = results_sd_7_n_500['Error'].mean()
sigma      = results_sd_7_n_500['Error'].std()
iterations = results_sd_7_n_500['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('10k_30k_50k_70k_90k_11k_13k_n500.png')
plt.clf()

In [ ]:
results_sd_7_n_1000 = infer_power([10000, 30000, 50000, 70000, 90000, 110000, 130000], 
                                  1000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_7_n_1000 complete.')
mu         = results_sd_7_n_1000['Error'].mean()
sigma      = results_sd_7_n_1000['Error'].std()
iterations = results_sd_7_n_1000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('10k_30k_50k_70k_90k_11k_13k_n1000.png')
plt.clf()

In [ ]:
results_sd_7_n_2000 = infer_power([10000, 30000, 50000, 70000, 90000, 110000, 130000], 
                                   2000, 100, 0.05, 0.8, 'data_c_13000_mde_01.csv')
print('results_sd_7_n_2000 complete.')
mu         = results_sd_7_n_2000['Error'].mean()
sigma      = results_sd_7_n_2000['Error'].std()
iterations = results_sd_7_n_2000['Iterations'].unique()[0]
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.axvline(x=0, color = 'red', alpha=0.4)
plt.xlim(-0.1,1)
bbox = dict(boxstyle="round", fc="0.9")
plt.annotate('Mean Error  = {} \n   Error Std. = {} \n   Iterations = {}'.format(round(mu,2), 
                                                                                 round(sigma,2),
                                                                                 iterations),
            (xdisplay, ydisplay), xytext=(257, 200),
            xycoords='figure pixels',
            textcoords='offset points',
            bbox=bbox)
# plt.show()
plt.savefig('10k_30k_50k_70k_90k_11k_13k_n2000.png')
plt.clf()